In [1]:
import cv2
import numpy as np
import json
import time

In [2]:
start_frame = 0

In [9]:
TrDict = {
  # 'csrt': cv2.legacy.TrackerCSRT_create,
  # 'kcf' : cv2.legacy.TrackerKCF_create,
  # 'boosting' : cv2.legacy.TrackerBoosting_create,
  # 'mil': cv2.legacy.TrackerMIL_create,
  # 'tld': cv2.legacy.TrackerTLD_create,
  'medianflow': cv2.legacy.TrackerMedianFlow_create,
  'mosse':cv2.legacy.TrackerMOSSE_create
  }

In [ ]:
video_names = 'uid_vid_00000.mp4'

In [5]:
v = cv2.VideoCapture(f'../data/{video_name}')

In [6]:
with open(f'../annotation/{video_name}.json') as f:
    annotation = json.load(f)

In [ ]:
gt_data = {}
for entity in annotation["entities"]:
    frame_idx = entity["blob"]["frame_idx"]
    bbox = entity["bb"]

    if frame_idx not in gt_data:
        gt_data[frame_idx] = []

    if 'person' in entity["labels"].keys():
        gt_data[frame_idx].append(bbox)

gt_data

{24: [[506, 881, 112, 172],
  [106, 555, 36, 172],
  [172, 410, 64, 199],
  [138, 702, 86, 227],
  [190, 608, 60, 181],
  [290, 527, 57, 190],
  [104, 447, 56, 186],
  [342, 618, 63, 190],
  [377, 588, 69, 176],
  [481, 596, 90, 190],
  [1380, 142, 77, 166],
  [1194, 274, 71, 159],
  [1079, 254, 74, 180],
  [893, 352, 51, 173],
  [934, 374, 59, 157],
  [759, 481, 67, 109],
  [780, 452, 60, 106]],
 29: [[494, 886, 124, 164],
  [105, 555, 43, 170],
  [175, 416, 60, 195],
  [136, 692, 90, 216],
  [206, 613, 58, 193],
  [297, 532, 57, 201],
  [105, 447, 59, 189],
  [345, 617, 112, 191],
  [391, 585, 59, 182],
  [497, 598, 92, 185],
  [1391, 132, 71, 167],
  [1200, 265, 77, 162],
  [1088, 248, 74, 186],
  [901, 348, 56, 176],
  [945, 370, 58, 160],
  [780, 478, 61, 99],
  [798, 439, 65, 111]],
 34: [[507, 885, 114, 167],
  [106, 555, 44, 170],
  [181, 416, 64, 196],
  [133, 687, 88, 222],
  [216, 628, 73, 199],
  [305, 541, 59, 192],
  [106, 447, 60, 189],
  [382, 612, 78, 190],
  [417, 584

In [8]:
frames_to_track = sorted(gt_data.keys())
frames_to_track

[0,
 5,
 10,
 14,
 19,
 24,
 29,
 34,
 38,
 43,
 48,
 53,
 58,
 62,
 67,
 72,
 77,
 82,
 86,
 91,
 96,
 101,
 106,
 110,
 115,
 120,
 125,
 129,
 134,
 139,
 144,
 149,
 153,
 158,
 163,
 168,
 173,
 177,
 182,
 187,
 192,
 197,
 201,
 206,
 211,
 216,
 221,
 225,
 230,
 235,
 240,
 245,
 249,
 254,
 259,
 264,
 269,
 273,
 278,
 283,
 288,
 293,
 297,
 302,
 307,
 312,
 317,
 321,
 326,
 331,
 336,
 341,
 345,
 350,
 355,
 360,
 364,
 369,
 374,
 379,
 384,
 388,
 393,
 398,
 403,
 408,
 412,
 417,
 422,
 427]

In [9]:
objs_to_track = []
for entity in annotation["entities"]:
    if entity["blob"]["frame_idx"] == start_frame:
        if 'person' in entity["labels"].keys():
            if entity["id"] not in objs_to_track:
                objs_to_track.append(entity["id"])

objs_to_track

[1, 22, 23, 26, 27, 29, 30, 33, 35, 36, 37, 38, 39, 41, 42]

In [10]:
tracker_boxes = {frame_idx: [] for frame_idx in frames_to_track}
tracker_boxes

{0: [],
 5: [],
 10: [],
 14: [],
 19: [],
 24: [],
 29: [],
 34: [],
 38: [],
 43: [],
 48: [],
 53: [],
 58: [],
 62: [],
 67: [],
 72: [],
 77: [],
 82: [],
 86: [],
 91: [],
 96: [],
 101: [],
 106: [],
 110: [],
 115: [],
 120: [],
 125: [],
 129: [],
 134: [],
 139: [],
 144: [],
 149: [],
 153: [],
 158: [],
 163: [],
 168: [],
 173: [],
 177: [],
 182: [],
 187: [],
 192: [],
 197: [],
 201: [],
 206: [],
 211: [],
 216: [],
 221: [],
 225: [],
 230: [],
 235: [],
 240: [],
 245: [],
 249: [],
 254: [],
 259: [],
 264: [],
 269: [],
 273: [],
 278: [],
 283: [],
 288: [],
 293: [],
 297: [],
 302: [],
 307: [],
 312: [],
 317: [],
 321: [],
 326: [],
 331: [],
 336: [],
 341: [],
 345: [],
 350: [],
 355: [],
 360: [],
 364: [],
 369: [],
 374: [],
 379: [],
 384: [],
 388: [],
 393: [],
 398: [],
 403: [],
 408: [],
 412: [],
 417: [],
 422: [],
 427: []}

In [11]:
start_bounding_boxes = gt_data[start_frame]
start_bounding_boxes

[[511, 889, 135, 165],
 [105, 400, 73, 201],
 [101, 752, 105, 224],
 [105, 586, 105, 185],
 [240, 504, 53, 114],
 [230, 604, 65, 184],
 [280, 565, 55, 183],
 [388, 604, 98, 190],
 [1318, 172, 85, 169],
 [1141, 299, 62, 164],
 [1029, 291, 76, 190],
 [816, 376, 65, 171],
 [867, 408, 64, 112],
 [659, 526, 65, 143],
 [700, 488, 57, 126]]

In [12]:
trackers = cv2.legacy.MultiTracker_create()

In [13]:
tracking_results = {tracker: {} for tracker in TrDict.keys()}
tracking_results

{'csrt': {}}

In [14]:
for tracker in TrDict:
  print(f"Using tracker: {tracker}")

  start_time = time.time()
  tracker_boxes[start_frame] = start_bounding_boxes
  
  v.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
  ret, frame = v.read()

  for bbi in start_bounding_boxes:
    tracker_i = TrDict[tracker]()
    trackers.add(tracker_i, frame, bbi)

  while True:
    ret, frame = v.read()
    if not ret:
      break

    frame_idx = int(v.get(cv2.CAP_PROP_POS_FRAMES))

    (success,boxes) = trackers.update(frame)
    for box in boxes:
        (x,y,w,h) = [int(a) for a in box]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
        if frame_idx in frames_to_track:
          tracker_boxes[frame_idx].append((x,y,w,h))
        
    
    cv2.imshow('Frame',frame)
    key = cv2.waitKey(5) & 0xFF
    if key == ord('q'):
        break
    
  elapsed_time = time.time() - start_time
  tracking_results[tracker]['bounding_boxes'] = tracker_boxes
  tracking_results[tracker]['time'] = elapsed_time

cv2.destroyAllWindows()
v.release()

Using tracker: csrt


2025-04-01 20:47:47.486 python[13642:5064138] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-01 20:47:47.486 python[13642:5064138] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [15]:
tracking_results['csrt']['bounding_boxes']

{0: [[511, 889, 135, 165],
  [105, 400, 73, 201],
  [101, 752, 105, 224],
  [105, 586, 105, 185],
  [240, 504, 53, 114],
  [230, 604, 65, 184],
  [280, 565, 55, 183],
  [388, 604, 98, 190],
  [1318, 172, 85, 169],
  [1141, 299, 62, 164],
  [1029, 291, 76, 190],
  [816, 376, 65, 171],
  [867, 408, 64, 112],
  [659, 526, 65, 143],
  [700, 488, 57, 126]],
 5: [(511, 889, 135, 165),
  (113, 401, 73, 201),
  (107, 744, 107, 228),
  (116, 598, 99, 174),
  (248, 507, 52, 112),
  (248, 617, 61, 173),
  (292, 562, 61, 202),
  (404, 603, 100, 194),
  (1324, 163, 87, 172),
  (1151, 294, 63, 167),
  (1037, 285, 78, 194),
  (823, 372, 68, 178),
  (874, 399, 72, 126),
  (675, 514, 69, 152),
  (714, 477, 59, 131)],
 10: [(512, 889, 135, 165),
  (127, 402, 67, 186),
  (116, 738, 105, 224),
  (131, 608, 84, 149),
  (259, 515, 51, 110),
  (269, 630, 64, 180),
  (314, 587, 53, 176),
  (445, 615, 89, 172),
  (1337, 158, 87, 172),
  (1167, 293, 62, 164),
  (1042, 272, 78, 194),
  (840, 364, 68, 178),
  (89

In [16]:
results = {tracker: {} for tracker in TrDict.keys()}

for tracker in TrDict:
    results[tracker] = {
        'entities': [],
        'metadata': annotation['metadata'],
    }

    for frame_idx, boxes in tracking_results[tracker]['bounding_boxes'].items():
        for i, box in enumerate(boxes):
            obj_id = objs_to_track[i]
            entity = {
                'bb': box,
                'blob': {
                    'frame_idx': frame_idx,
                },
                'confidence': 1.0,
                'id': obj_id,
                'labels': {
                    'person': 1,
                },
                'time': 1000
            }
            results[tracker]['entities'].append(entity)


In [ ]:
with open(f'../results/{video_name}.json', 'w') as f:
    json.dump(results, f, indent=4)

: 